## Introduction   
In this notebook we introduce the basics of blocks and views.  The reader should first be familiar with the C VSIPL specification introduction chapter so they are familiar with how blocks and views work. Under the covers pyJvsip blocks and views are based on C VSIPL blocks and views.

In [18]:
import pyJvsip as pjv
f='%.2f' #used for formating mprint

### Block  
Creating a block requires a block type and the number of elements in the block. The memory flag is not used in pyJvsip. The pyJvsip Block object has a method to return the number of elements in the block. This method is not supported by the C VSIPL block object.   

Memory flag defaults to VSIP_MEM_NONE. Note that C VSIPL does not support hints like memory flag or algorithm except the API is supported, so pyJvsip will frequently default these to some value.

In [19]:
aFloatBlock=pjv.Block('block_f',100)
aDoubleBlock=pjv.Block('block_d',100)
aComplexDoubleBlock=pjv.Block('block_d',100)
aIntBlock=pjv.Block('block_i',100)

In [20]:
aFloatBlock.length

100

### View
A pyJvsip view is always associated with a block. All views are created with a block bind method.    
   
The number of arguments to bind are used to determine if a matrix or vector view are created.    
A vector view is bind(offset,stride, length)   
A matrix view is bind(offset, columnStride, columnLength, rowStride, rowLength)

In [21]:
aFloatVectorView=aFloatBlock.bind(0,2,10)
aFloatVectorView.ramp(1,1)
aFloatVectorView.mprint(f)

[ 1.00  2.00  3.00  4.00  5.00  6.00  7.00  8.00  9.00  10.00]



In [26]:
aDoubleMatrixView=aDoubleBlock.bind(0,5,6,1,5)

In [27]:
for i in range(aDoubleMatrixView.collength):
    aDoubleMatrixView.rowview(i).ramp(i,1)
aDoubleMatrixView.mprint(f)

[ 0.00  1.00  2.00  3.00  4.00;
  1.00  2.00  3.00  4.00  5.00;
  2.00  3.00  4.00  5.00  6.00;
  3.00  4.00  5.00  6.00  7.00;
  4.00  5.00  6.00  7.00  8.00;
  5.00  6.00  7.00  8.00  9.00]



A vector view has a block property so the block a view is associated with can always be recovered and used to make another view on the same block.

In [28]:
aNewVector = aFloatVectorView.block.bind(0,1,10)
aNewVector.mprint(f)

[ 1.00 -8589934592.00  2.00 -8589934592.00  3.00  0.00  4.00 -54913055254904832.00  5.00  0.00]



Above note we created the block and placed a view on it with stride two and then put a ramp in the view. Here we have created a new view on the same data space with stride one. But we did not initialize this vector so it has coresponding values from the first vector pluss mystery values from our uninitialized block.

Sometimes you want a vector on an entire block; perhaps to initialize the block values. For convenience pyJvsip supplies a block method to do this for you.

In [29]:
aFloatVectorView.block.vector.fill(0.0)
aFloatVectorView.ramp(1,1)
aFloatVectorView.mprint(f)
aNewVector.mprint(f)

[ 1.00  2.00  3.00  4.00  5.00  6.00  7.00  8.00  9.00  10.00]

[ 1.00  0.00  2.00  0.00  3.00  0.00  4.00  0.00  5.00  0.00]



Note above we didn't care about the vector on the whole block so we didn't create a reference to it. pyJvsip uses python's garbage collection methods to destroy pyJvsip objects which have no reference.  We can see now the block has been initialized so our second vector has a zero in every other spot. 

## Create Function   

The pyJvsip module has a create function which may be used to create pyJvsip objects including blocks and views. Most of the time you will use this function. When a view is created with this function it creates the needed block under the covers for you.

In [30]:
aVector=pjv.create('vview_d',10)
aVector.ramp(1,1)
aVector.mprint(f)

[ 1.00  2.00  3.00  4.00  5.00  6.00  7.00  8.00  9.00  10.00]



Be default when creating a matrix view the view is created as row major.

In [31]:
aMatrix = pjv.create('cmview_d',4,5)
aMatrix.block.vector.ramp(1,1)
aMatrix.imagview.block.vector.ramp(-1,-1)
aMatrix.mprint('%.1f')

[ 1.0-1.0i  2.0-2.0i  3.0-3.0i  4.0-4.0i  5.0-5.0i;
  6.0-6.0i  7.0-7.0i  8.0-8.0i  9.0-9.0i  10.0-10.0i;
  11.0-11.0i  12.0-12.0i  13.0-13.0i  14.0-14.0i  15.0-15.0i;
  16.0-16.0i  17.0-17.0i  18.0-18.0i  19.0-19.0i  20.0-20.0i]



We can also add a major argument if we want a column major matrix

In [32]:
aMatrix = pjv.create('cmview_d',4,5,'COL')
aMatrix.block.vector.ramp(1,1)
aMatrix.imagview.block.vector.ramp(-1,-1)
aMatrix.mprint('%.1f')

[ 1.0-1.0i  5.0-5.0i  9.0-9.0i  13.0-13.0i  17.0-17.0i;
  2.0-2.0i  6.0-6.0i  10.0-10.0i  14.0-14.0i  18.0-18.0i;
  3.0-3.0i  7.0-7.0i  11.0-11.0i  15.0-15.0i  19.0-19.0i;
  4.0-4.0i  8.0-8.0i  12.0-12.0i  16.0-16.0i  20.0-20.0i]



There are other ways to create views using subviews. Note a subview is a first class view and there is no material difference between a subview and it's parent. See the VSIPL spec for more information on the relationship of a parent complex view and it's child real and imag views. A few examples. Of course you can always recover the block and do a blockbind if you want a view of the same depth.  To get a real or imaginary view of complex aparent you must always use the realview and imagview method. 

In [33]:
aMatrix.diagview(0).mprint(f)
aMatrix.realview.mprint(f)
aMatrix.rowview(2).mprint(f)
aMatrix.colview(1)[1:].mprint(f)
aMatrix[1:,2:].mprint(f)

[ 1.00-1.00i  6.00-6.00i  11.00-11.00i  16.00-16.00i]

[ 1.00  5.00  9.00  13.00  17.00;
  2.00  6.00  10.00  14.00  18.00;
  3.00  7.00  11.00  15.00  19.00;
  4.00  8.00  12.00  16.00  20.00]

[ 3.00-3.00i  7.00-7.00i  11.00-11.00i  15.00-15.00i  19.00-19.00i]

[ 6.00-6.00i  7.00-7.00i  8.00-8.00i]

[ 10.00-10.00i  14.00-14.00i  18.00-18.00i;
  11.00-11.00i  15.00-15.00i  19.00-19.00i;
  12.00-12.00i  16.00-16.00i  20.00-20.00i]



The last two methods demonstrate slicing. Currently pyJvsip matrix and vector views support some slicing methods but only for positive index values. 

In [34]:
aVector=pjv.create('vview_i',100).ramp(1,1)

In [35]:
aVector.mprint('%d')

[ 1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100]



In [36]:
aVector[10:20:2].mprint('%d')
aNewVector=aVector[4:30:3].copy
aNewVector.mprint('%d')

[ 11  13  15  17  19]

[ 5  8  11  14  17  20  23  26  29]



Note the copy method above creates a new block and vector on the block.   
You can also use a slice to copy data between vectors.

In [37]:
aNewVector[:]=aVector[:aNewVector.length]
aNewVector.mprint('%d')
aNewVector[::2]=aVector[aNewVector.length:2 * aNewVector.length:2]
aNewVector.mprint('%d')

[ 1  2  3  4  5  6  7  8  9]

[ 10  2  12  4  14  6  16  8  18]

